In [1]:
import os
import sys
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import display
from typing import Tuple

sys.path.append(os.getcwd() + "/../")

In [2]:
import bfcnn
from bfcnn.utilities import load_image

In [3]:
# setup environment
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.compat.v1.enable_eager_execution()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [4]:
config = bfcnn.CONFIGS_DICT["resnet_color_1x6_bn_32x32_3x3_128x128_l1_relu"]

models = bfcnn.model_builder(config=config["model_denoise"])

denoiser_model = models.denoiser

2022-11-22 16:47:05,246 INFO model_denoiser.py:model_builder:58] building model with config [{'filters': 32, 'no_layers': 6, 'kernel_size': 7, 'block_kernels': [3, 3], 'block_filters': [32, 32], 'value_range': [0, 255], 'type': 'resnet', 'batchnorm': True, 'activation': 'relu', 'final_activation': 'tanh', 'input_shape': ['?', '?', 3], 'kernel_initializer': 'glorot_normal', 'kernel_regularizer': 'l1'}]
2022-11-22 16:47:05,297 INFO model_denoiser.py:model_builder:239] pyramid produces [1] scales
2022-11-22 16:47:05,298 INFO model_denoiser.py:model_builder:250] building per scale model
2022-11-22 16:47:05,298 INFO backbone_resnet.py:builder:87] building resnet backbone
2022-11-22 16:47:05,298 INFO backbone_resnet.py:builder:88] parameters not used: {'add_var': False, 'no_levels': 1, 'add_laplacian': True}


In [5]:
# base directory for test images
IMAGES_TEST_DIR = \
    pathlib.Path(os.path.abspath("")).parent.resolve() / "images" / "test"

# directory of kitti test images
KITTI_DIR = \
    IMAGES_TEST_DIR / "kitti"

# all the kitti test images
KITTI_IMAGES = \
    [img for img in (KITTI_DIR / "files").glob("*.png")]

RESOLUTION = (64, 64)


In [ ]:
img_path_iterator = iter(KITTI_IMAGES)
img_path = next(img_path_iterator)

# prepare image
img = load_image(img_path=img_path, target_size=RESOLUTION)

with tf.GradientTape() as g:
    x = img
    g.watch(x)
    y = denoiser_model(x)

j = g.jacobian(y, x)

print(j.shape)